In [1]:
import numpy as np
from pyscf import gto, scf, fci
from multiprocessing.pool import Pool
from numba import prange, jit, njit
#from numba.openmp import openmp_context as openmp

This version of Umrigar_for_fake_Spinorbitals_parallel exists to illustrate that the loops in the energy expressions do not have to be decomposed to realize spin orthogonality. The vector products should work by themselves.
$$
\begin{align}
    E_H[\gamma_1]  &=  \sum_a^M \sum_b^M n_a n_b \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  \tilde{\tilde{c}}_{a,\mu} \tilde{\tilde{c}}_{b,\nu} \tilde{\tilde{c}}_{a,\kappa} \tilde{\tilde{c}}_{b,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
    E_H[\gamma_1]  &=   \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M \left (  \sum_a^M  n_a  \tilde{\tilde{c}}_{a,\mu} \tilde{\tilde{c}}_{a,\kappa} \right ) \left ( \sum_b^M n_b \tilde{\tilde{c}}_{b,\nu}  \tilde{\tilde{c}}_{b,\lambda} \right )\left [ \mu \nu | \kappa \lambda \right ] \\
  E_H[\gamma_1]  &=   \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M \left (  \vec{c}_{\mu}^{T} \vec{c}_{\kappa} \right ) \left ( \vec{c}_{\nu}^{T} \vec{c}_{\lambda} \right )\left [ \mu \nu | \kappa \lambda \right ] \\
\end{align}
$$

In [2]:
def expand_matrix(P):
    Paa = P
    Pbb = P
    Pab = np.zeros(P.shape)
    Pba = np.zeros(P.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [3]:
# for jit and prange
@jit(nopython=True, parallel=True)
def ONERDMFT_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

In [4]:
# for spinorbitals
@jit(nopython=True, parallel=True)
def ONERDMFT_Umrigar_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M):
        for b in [b for b in range(0,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy
#                if not(a == b):


In [5]:
@jit(parallel=True)
def ONERDMFT_exchange_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [6]:
#@jit(parallel=True)
def ONERDMFT_exchange_energy_parallel_implicit_orthogonality(Fouridx, C, n):
    vec_prod_a = 0 
    vec_prod_b = 0
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for mu in range(0,M):
        for nu in range(0,M):
            for kappa in range(0,M):
                for lamda in range(0,M):
                    vec_prod_a = 0 
                    vec_prod_b = 0
                    for a in  range(0,M):
                        for b in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
                            vec_prod_a += C[mu,a]*C[kappa,a]
                            vec_prod_b += C[nu,b]*C[lamda,b]
                    print(mu,kappa,nu,lamda, vec_prod_a, vec_prod_b, energy)

                            


    return -1/2.*energy

In [7]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
# this basis has 2 functions for Helium
#mol.basis = "6-31g" 
#mol.basis = "ccpvdz"
mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

print(eri.shape)

## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()



(1, 1, 1, 1)
converged SCF energy = -2.84629209484258


np.float64(-2.846292094842575)

In [8]:
# harvest Hartree Fock quantities
C = mf.mo_coeff
h = mf.get_hcore()
N = mol.nelec[0]

P=np.matmul(C[:,0:N],C[:,0:N].T)
gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
occu, naturalC = np.linalg.eigh(gamma)

print(C.shape)

(1, 1)


In [9]:
# need to first make gamma and then expand into blockdiagonal so "fake" alpha and "beta" are seperated in this fashion
CE = expand_matrix(C)
gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
occu_aa, naturalC_aa = np.linalg.eigh(gamma)
occu = np.append(occu_aa, occu_aa)
naturalCE = expand_matrix(naturalC_aa)
naturalCTT = np.matmul(CE,naturalCE)

In [10]:
for i, n  in enumerate(occu):
    if n < 0:
        occu[i] = 0 
print(occu)

[1. 1.]


In [11]:
FCI = True
if FCI:
    fs = fci.FCI(mol, mf.mo_coeff)
    e, ci = fs.kernel(verbose=0)
FCIgamma = fci.direct_spin1.make_rdm1(ci, mf.mo_coeff.shape[0], mol.nelec)
FCIgamma = FCIgamma/2.0
FCIoccu, FCInaturalC = np.linalg.eigh(FCIgamma)
FCInaturalCTT = np.matmul(C,FCInaturalC)
FCInaturalCTTE = expand_matrix(FCInaturalCTT)
FCIoccuE = np.append(FCIoccu,FCIoccu)
print(FCInaturalCTTE)

[[1. 0.]
 [0. 1.]]


In [12]:
E_x = ONERDMFT_exchange_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
E_x2 = ONERDMFT_exchange_energy_parallel_implicit_orthogonality(eri, FCInaturalCTTE, FCIoccuE)
print(E_x, E_x2)

0 0 0 0 2.0 2.0 1.0562479844540422
0 0 0 1 2.0 0.0 1.0562479844540422
0 1 0 0 0.0 2.0 1.0562479844540422
0 1 0 1 0.0 0.0 1.0562479844540422
0 0 1 0 2.0 0.0 1.0562479844540422
0 0 1 1 2.0 2.0 2.1124959689080844
0 1 1 0 0.0 0.0 2.1124959689080844
0 1 1 1 0.0 2.0 2.1124959689080844
1 0 0 0 0.0 2.0 2.1124959689080844
1 0 0 1 0.0 0.0 2.1124959689080844
1 1 0 0 2.0 2.0 3.1687439533621267
1 1 0 1 2.0 0.0 3.1687439533621267
1 0 1 0 0.0 0.0 3.1687439533621267
1 0 1 1 0.0 2.0 3.1687439533621267
1 1 1 0 2.0 0.0 3.1687439533621267
1 1 1 1 2.0 2.0 4.224991937816169
-1.0562479844540422 -2.1124959689080844
